In [ ]:
# %matplotlib widget
import argparse
from pathlib import Path
from typing import List
from dataclasses import dataclass, asdict
from datetime import datetime

import matplotlib
matplotlib.use('Agg')  # Configure backend before importing pyplot
import matplotlib.pyplot as plt
import numpy as np
from threed_utils.multiview_calibration.detection import run_checkerboard_detection, detect_chessboard, plot_chessboard_qc_data, generate_chessboard_objpoints
from threed_utils.multiview_calibration.calibration import calibrate, get_intrinsics
from threed_utils.multiview_calibration.bundle_adjustment import bundle_adjust
from threed_utils.multiview_calibration.viz import plot_residuals, plot_shared_detections
from threed_utils.multiview_calibration.geometry import triangulate
from tqdm import tqdm, trange
import hickle
from threed_utils.io import write_calibration_toml
import cv2
from pipeline_params import CalibrationOptions, DetectionOptions, DetectionRunnerOptions, ProcessingOptions
from movement.io.load_poses import from_numpy
import xarray as xr
from threed_utils.detection_napari_check import view_movement_3d

from threed_utils.anipose.movement_anipose import anipose_triangulate_ds
from threed_utils.io import load_calibration
from threed_utils.arena_utils import load_arena_multiview_ds


In [ ]:
# Run detection
calibration_options = CalibrationOptions()
detection_runner_options = DetectionRunnerOptions()
detection_options = DetectionOptions()
n_workers = 1
folder = Path("/Users/vigji/Desktop/test_3d/Calibration/20250509/multicam_video_2025-05-09T09_56_51_cropped-v2_20250710121328")


all_calib_uvs, all_img_sizes, video_files_dict = run_checkerboard_detection(folder, 
                                                                        extension=detection_runner_options.video_extension, 
                                                                        overwrite=detection_runner_options.overwrite,
                                                                        detection_options=asdict(detection_options),
                                                                        n_workers=n_workers)

camera_names = list(video_files_dict.keys())
video_paths = list(video_files_dict.values())
print(f"Found cameras: {camera_names}")


def uvs_to_ds(all_calib_uvs, camera_names):
    all_xarrs = []
    for all_calib_pts in all_calib_uvs:
        reshaped_for_mov = np.swapaxes(all_calib_pts, 1, 2)[:, :, :, None]
        mov_xarr = from_numpy(reshaped_for_mov, individual_names=["calibrator"], keypoint_names=np.arange(reshaped_for_mov.shape[2]),
                            confidence_array=np.ones_like(reshaped_for_mov[:, 0, :, :]))
        all_xarrs.append(mov_xarr)
    new_coord_views = xr.DataArray(camera_names, dims="view")
    return xr.concat(all_xarrs, dim=new_coord_views)

uvs_ds = uvs_to_ds(all_calib_uvs, camera_names)

valid_uvs_ds_idxs = (~np.isnan(uvs_ds.position.values[:, :, 0, 0, 0])).sum(axis=0) > 1  #.sum(0) > 1
valid_uvs_ds = uvs_ds.isel(time=valid_uvs_ds_idxs)

In [ ]:
arena_json = Path("/Users/vigji/Desktop/test_3d/multicam_video_2025-05-07T10_12_11_20250528-153946.json")
arena_ds = load_arena_multiview_ds(arena_json)


In [ ]:
calib_dir = folder 
calib_dir.exists()

all_calibvals = load_calibration(calib_dir)

In [ ]:
calib_toml = folder / "mc_calibration_output_20250710-152443" / "calibration_from_mc.toml"
assert calib_toml.exists()
triang_config_optim = {
    "ransac": True,
    "optim": False,}
checkerboard_triang_ds = anipose_triangulate_ds(views_ds=valid_uvs_ds, calib_toml_path=calib_toml, **triang_config_optim)
# stack arena_ds over time 
arena_triang_ds = anipose_triangulate_ds(views_ds=arena_ds, calib_toml_path=calib_toml, **triang_config_optim)
arena_triang_ds = xr.concat([arena_triang_ds,]*len(valid_uvs_ds.coords["time"]), dim="time")

In [ ]:
viewer = napari.Viewer()
view_movement_3d(checkerboard_triang_ds, viewer=viewer)
view_movement_3d(arena_triang_ds, viewer=viewer)